# Traveler's Dilemma
Simple Game

*Ghi chú:* **Các giải thuật dưới đây là những lời giải tốt nhất tìm được để chứng minh cho kết luận của bài toán. Các giải thuật khác đã thử nghiệm và kết quả của chúng được ghi trong báo cáo.**

## Resource
Add the necessary packages and declare some useful functions.

In [ ]:
import Pkg

function addPackage(pkg::String)
    if !haskey(Pkg.installed(), pkg)
        Pkg.add(pkg)
    end
end

addPackage("Distributions")
addPackage("LinearAlgebra")
addPackage("JuMP")
addPackage("Ipopt")

using Distributions, LinearAlgebra, JuMP, Ipopt, Random


# Appendices
# G.5 Convenience Functions
struct SetCategorical{S}
    elements::Vector{S} # Set elements (could be repeated)
    distr::Categorical # Categorical distribution over set elements

    function SetCategorical(elements::AbstractVector{S}) where {S}
        weights = ones(length(elements))
        return new{S}(elements, Categorical(normalize(weights, 1)))
    end

    function SetCategorical(elements::AbstractVector{S}, weights::AbstractVector{Float64}) where {S}
        ℓ₁ = norm(weights, 1)
        if ℓ₁ < 1e-6 || isinf(ℓ₁)
            return SetCategorical(elements)
        end
        distr = Categorical(normalize(weights, 1))
        return new{S}(elements, distr)
    end
end

Distributions.rand(D::SetCategorical) = D.elements[rand(D.distr)]
Distributions.rand(D::SetCategorical, n::Int) = D.elements[rand(D.distr, n)]

function Distributions.pdf(D::SetCategorical, x)
    sum(e == x ? w : 0.0 for (e, w) in zip(D.elements, D.distr.p))
end


## Define simple game structures 

In [ ]:
# Algorithm 24.1. Data structure for a simple game.
struct SimpleGame
    γ # discount factor
    ℐ # agents
    𝒜 # joint action space
    R # joint reward function
end


# Algorithm 24.2
struct SimpleGamePolicy
    p # dictionary mapping actions to probabilities

    function SimpleGamePolicy(p::Base.Generator)
        return SimpleGamePolicy(Dict(p))
    end

    function SimpleGamePolicy(p::Dict)
        vs = collect(values(p))
        vs ./= sum(vs)
        return new(Dict(k => v for (k, v) in zip(keys(p), vs)))
    end

    SimpleGamePolicy(ai) = new(Dict(ai => 1.0))
end

(πi::SimpleGamePolicy)(ai) = get(πi.p, ai, 0.0)

function (πi::SimpleGamePolicy)()
    D = SetCategorical(collect(keys(πi.p)), collect(values(πi.p)))
    return rand(D)
end

joint(X) = vec(collect(Iterators.product(X...)))

joint(π, πi, i) = [i == j ? πi : πj for (j, πj) in enumerate(π)] # helper of best_response

function utility(𝒫::SimpleGame, π, i)
    𝒜, R = 𝒫.𝒜, 𝒫.R
    p(a) = prod(πj(aj) for (πj, aj) in zip(π, a))
    return sum(R(a)[i] * p(a) for a in joint(𝒜))
end


# Algorithm 24.3
function best_response(𝒫::SimpleGame, π, i)
    U(ai) = utility(𝒫, joint(π, SimpleGamePolicy(ai), i), i)
    ai = argmax(U, 𝒫.𝒜[i])
    return SimpleGamePolicy(ai)
end

## Problem properties
Register the properties of Traveler's Dilemma problem.

In [ ]:
const N_AGENTS = 2
ACTIONS = vec(collect(2:100))

function joint_reward(a::Tuple{Int64,Int64})
    ai, aj = a
    return ai == aj ? (ai, aj) : (ai < aj ? (ai + 2, ai - 2) : (aj - 2, aj + 2))
end

travelersDilemma = SimpleGame(
    1.0,
    vec(collect(1:N_AGENTS)),
    [ACTIONS for _ in 1:N_AGENTS],
    joint_reward)

## Hierarchical Softmax solution

In [ ]:
# Algorithm 24.4
function softmax_response(𝒫::SimpleGame, π, i, λ)
    𝒜i = 𝒫.𝒜[i]
    U(ai) = utility(𝒫, joint(π, SimpleGamePolicy(ai), i), i)
    return SimpleGamePolicy(ai => exp(λ * U(ai)) for ai in 𝒜i)
end


# Algorithm 24.9
struct HierarchicalSoftmax
    λ # precision parameter
    k # level
    π # initial policy
end

function HierarchicalSoftmax(𝒫::SimpleGame, λ, k)
    π = [SimpleGamePolicy(ai => 1.0 for ai in 𝒜i) for 𝒜i in 𝒫.𝒜]
    return HierarchicalSoftmax(λ, k, π)
end

function solve(M::HierarchicalSoftmax, 𝒫)
    π = M.π
    for k in 1:M.k
        π = [softmax_response(𝒫, π, i, M.λ) for i in 𝒫.ℐ]
    end
    return π
end


π = solve(HierarchicalSoftmax(travelersDilemma, 0.3, 4), travelersDilemma)

π¹ = π[1].p
π² = π[2].p

for a in ACTIONS
    println(a => (π¹[a], π²[a]))
end

## Reduce the number of actions available
Reduce the number of actions available from \\$100 to \\$20 maximum value can be put down for saving runtime. It is easy to see that this does not greatly affect the conclusion of the problem.

In [ ]:
ACTIONS = vec(collect(2:20))

travelersDilemma = SimpleGame(
    1.0,
    vec(collect(1:N_AGENTS)),
    [ACTIONS for _ in 1:N_AGENTS],
    joint_reward)

## Correlated Equilibrium solution
General of Nash equilibrium concept by relaxing the assumption that the agents act independently.

In [ ]:
# Algorithm 24.6
mutable struct JointCorrelatedPolicy
    p # dictionary mapping from joint actions to probabilities
    JointCorrelatedPolicy(p::Base.Generator) = new(Dict(p))
end

(π::JointCorrelatedPolicy)(a) = get(π.p, a, 0.0)

function (π::JointCorrelatedPolicy)()
    D = SetCategorical(collect(keys(π.p)), collect(values(π.p)))
    return rand(D)
end


# Algorithm 24.7 (Utilitarian) [Fixed bug by me]
struct CorrelatedEquilibrium end

joint(a, ai′, i) = Tuple(k == i ? ai′ : v for (k, v) in enumerate(a))

function solve(M::CorrelatedEquilibrium, 𝒫::SimpleGame)
    ℐ, 𝒜, R = 𝒫.ℐ, 𝒫.𝒜, 𝒫.R
    model = Model(Ipopt.Optimizer)
    @variable(model, π[joint(𝒜)] ≥ 0)
    @objective(model, Max, sum(sum(π[a] * R(a)[i] for a in joint(𝒜)) for i in ℐ))
    @constraint(model, [i = ℐ, ai = 𝒜[i], ai′ = 𝒜[i]],
        sum(R(a)[i] * π[a] for a in joint(𝒜) if a[i] == ai)
        ≥
        sum(R(joint(a, ai′, i))[i] * π[a] for a in joint(𝒜) if a[i] == ai))
    @constraint(model, sum(π) == 1)
    optimize!(model)
    return JointCorrelatedPolicy(a => value(π[a]) for a in joint(𝒜))
end


π = solve(CorrelatedEquilibrium(), travelersDilemma)

π¹ = Dict(a => 0.0 for a in travelersDilemma.𝒜[1])
π² = Dict(a => 0.0 for a in travelersDilemma.𝒜[2])

for (k, v) in π.p
    π¹[k[1]] += v
    π²[k[2]] += v
end

for a in ACTIONS
    println(a => (π¹[a], π²[a]))
end

## Fictitious Play solution

In [ ]:
# Algorithm 24.11
mutable struct FictitiousPlay
    𝒫 # simple game
    i # agent index
    N # array of action count dictionaries
    πi # current policy
end

function FictitiousPlay(𝒫::SimpleGame, i)
    N = [Dict(aj => 1 for aj in 𝒫.𝒜[j]) for j in 𝒫.ℐ]
    πi = SimpleGamePolicy(ai => 1.0 for ai in 𝒫.𝒜[i])
    return FictitiousPlay(𝒫, i, N, πi)
end

(πi::FictitiousPlay)() = πi.πi()

(πi::FictitiousPlay)(ai) = πi.πi(ai)

function update!(πi::FictitiousPlay, a)
    N, 𝒫, ℐ, i = πi.N, πi.𝒫, πi.𝒫.ℐ, πi.i
    for (j, aj) in enumerate(a)
        N[j][aj] += 1
    end
    p(j) = SimpleGamePolicy(aj => u / sum(values(N[j])) for (aj, u) in N[j])
    π = [p(j) for j in ℐ]
    πi.πi = best_response(𝒫, π, i)
end


# Algorithm 24.10
function simulate(𝒫::SimpleGame, π, k_max)
    for k = 1:k_max
        a = [πi() for πi in π]
        for πi in π
            update!(πi, a)
        end
    end
    return π
end


for k_max in [100, 1000, 10000, 100000]
    π = simulate(
        travelersDilemma,
        [FictitiousPlay(travelersDilemma, i) for i in travelersDilemma.ℐ],
        k_max)

    println("After ", k_max, " iterations, the (deterministic) policy:")
    
    π¹ = π[1].πi
    π² = π[2].πi
    
    println("π¹ = ", π¹)
    println("π² = ", π²)
    println()
end